In [47]:
import os
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext

In [48]:
os.getcwd()

'/home/jovyan/work'

In [49]:
dir = os.path.join(os.getcwd(), 'learning_spark_data') #실제 자기의 폴더????
dir

'/home/jovyan/work/learning_spark_data'

In [50]:
conf = SparkConf().setMaster('local').setAppName('mnm_practice')

In [51]:
#conf 대신 seesion 쓰는게 편함 
spark = SparkContext(conf = conf).getOrCreate()
spark

<SparkContext master=local appName=mnm_practice>

In [52]:
filename = 'mnm_dataset.csv'
filepath = os.path.join(dir, filename)
filepath

'/home/jovyan/work/learning_spark_data/mnm_dataset.csv'

In [53]:
rdd = spark.textFile('file:///'+filepath.replace('\\','/')) #실패이유 : 경로 못 알아들음 

In [13]:
rdd.count()

100000

In [16]:
#데이터 확인 
rdd.take(5)

['State,Color,Count', 'TX,Red,20', 'NV,Blue,66', 'CO,Blue,79', 'OR,Blue,71']

In [14]:
header = rdd.first()
header

'State,Color,Count'

In [20]:
# filer: header 제거, map: 원하는 필드 추출 동시에 
# map(lambda line: line.split(",") : 각 줄을 쉼표 기준으로 나눠, 그 다음 리스트로 바뀜
# RDD[List[Str]] 형태로 바뀜 
datas = rdd.filter(lambda row : row != header)\
            .map(lambda line: line.split(","))
datas

PythonRDD[12] at RDD at PythonRDD.scala:53

In [21]:
datas.take(5)

[['TX', 'Red', '20'],
 ['NV', 'Blue', '66'],
 ['CO', 'Blue', '79'],
 ['OR', 'Blue', '71'],
 ['WA', 'Yellow', '93']]

In [22]:
# 총 사탕 수 몇개일까 ?
# sum 기능이 없어서 만들어 줘야함 

In [39]:
# data 변환 -> 함침 
# 주 당 몇개씩 팔렸는지 
# ('CA', 20) 이러한 형태로 변환
rdd1 = datas.map(lambda x: ( x[0], int(x[2])))
rdd1.take(3)
# 같은 주끼리 값을 합침 
# 두개의 행을 받아서 같은 키끼리 합쳐 
total_by_state = rdd1.reduceByKey(lambda a,b: a+b)
total_by_state.take(3)

[('TX', 556493), ('NV', 548477), ('CO', 556302)]

In [40]:
# state. colour count 
# 주와 색상을 함께 묶어서 각 색상의 사탕의 개수를 주별로 합산 
rdd3 = datas.map(lambda x: ((x[0], x[1]), int(x[2])))\
    .reduceByKey(lambda a,b: a+b)
rdd3.take(3)

[(('TX', 'Red'), 95404), (('NV', 'Blue'), 90003), (('CO', 'Blue'), 93412)]

In [41]:
# 전체 목록 보기 > 모두 메모리에 올린다 
rdd3.collect()

[(('TX', 'Red'), 95404),
 (('NV', 'Blue'), 90003),
 (('CO', 'Blue'), 93412),
 (('OR', 'Blue'), 90526),
 (('WA', 'Yellow'), 92920),
 (('WY', 'Blue'), 91002),
 (('CA', 'Yellow'), 100956),
 (('WA', 'Green'), 96486),
 (('OR', 'Green'), 89578),
 (('TX', 'Green'), 95753),
 (('NV', 'Green'), 91331),
 (('AZ', 'Brown'), 92287),
 (('AZ', 'Blue'), 89971),
 (('OR', 'Brown'), 89136),
 (('NV', 'Red'), 89346),
 (('WY', 'Orange'), 87956),
 (('TX', 'Brown'), 90736),
 (('CO', 'Red'), 89465),
 (('AZ', 'Green'), 91882),
 (('NM', 'Yellow'), 92747),
 (('WA', 'Red'), 93332),
 (('WY', 'Red'), 91768),
 (('TX', 'Orange'), 92315),
 (('WY', 'Yellow'), 87800),
 (('CO', 'Orange'), 90971),
 (('NV', 'Orange'), 93929),
 (('WY', 'Green'), 94339),
 (('CA', 'Green'), 93505),
 (('UT', 'Red'), 90995),
 (('CA', 'Red'), 91527),
 (('TX', 'Yellow'), 93819),
 (('WA', 'Orange'), 91521),
 (('AZ', 'Orange'), 91684),
 (('AZ', 'Red'), 90042),
 (('CA', 'Blue'), 89123),
 (('CO', 'Brown'), 93692),
 (('UT', 'Green'), 88392),
 (('NM', 'G

In [37]:
# output으로 내보내기 

In [ ]:
# 이걸 그대로 내보냈을떄와 mysql에 저장했을떄 어떻게 다른지 

In [38]:
# 저장소에 저장하기
output_path = 'output/mnm_result'
import shutil
if os.path.exists(output_path):
    shutil.rmtree(output_path)
rdd3.saveAsTextFile(output_path) #저장소 경로

In [43]:
!python 03_mnm_result_load.py

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/04 01:50:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/08/04 01:50:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
결과 미리보기 ------
Traceback (most recent call last):
  File "/home/jovyan/work/03_mnm_result_load.py", line 8, in <module>
    for line in result_rdd.take(5):
                ^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/pyspark/rdd.py", line 2822, in take
    totalParts = self.getNumPartitions()
                 ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/pyspark/rdd.py", line 952, in getNumPartitions
    return self._jrdd.partitions().size()
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1322, in __call__
  File "/usr/local/spark/py

In [44]:
spark.stop()